In [1]:
import os,re,csv,sys,ast,json,time,urllib,random,spotipy,requests,datetime
import numpy as np
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup
from user_agent import generate_user_agent
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# 原始所有歌手的清單 (透過collectArtists.ipnb整理)
df_artists = pd.read_excel(r'csv/20200303/v3/artist_list_v3.xlsx')
df_artists

,Artist
0,A Girl Called Eddy
1,A. A. Bondy
2,Aaliyah
3,Aaron Lee Tasjan
4,Abimbola Fernandez
...,...
1917,Zakiya
1918,Zella Day
1919,Zendaya
1920,Zola Taylor


In [3]:
# spotifyAPI存取權限
# access token要隨時更新 (https://spotify-token-finder.herokuapp.com/)
client_id = '1a23ed1ca359485ab81d524206dba0f6'
client_secret = 'af0e14fdb19a49dfb19ef0c1ea1d42ea'
username = 'qnwv65t11cplaz4dikhl4mjgi'
access_token = 'BQB9YYBL2Hus13pjPJKPcYnILH1Ydmwi2LZIgabKEUwUcqeK7I0-EVM1K3r9H80t-ZOCfA5ZveVUmJBEQEI'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# 取得歌手資訊
def get_artist(name):
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    # 歌手有items資訊
    if items:
        data = items[0]
        if len(items) > 0:
            artist_name = items[0]['name']
            artist_id = items[0]['id']
            artist_url = items[0]['external_urls']['spotify']
            artist_info = (artist_name,artist_id,artist_url)
            #print(artist_info)
            return items[0]
        else:
            return None
    # 歌手沒有items資訊，則回傳check值為1
    else:
        print('None items')
        check = 1
        return check
        pass

In [5]:
# 取得歌手名稱和類型
# https://github.com/Leats/spotify-historymaker/blob/master/spotifyhistorymaker.py
# Bruno Mars = 0du5cEVh5yTK9QJze8zA0C / Adele = 4dpARuHxo51G3z768sgnrY
def get_artist_name_genres(access_token: str, artist_id: str):    
    bearer_token = f'Bearer {access_token}'
    track_headers = {'Accept': 'application/json','Content-Type': 'application/json','Authorization': bearer_token}
    response = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=track_headers)
    response.raise_for_status()
    getname = response.json()['name'] # 型態str
    getid = response.json()['id'] # 型態str
    getgenres = response.json()['genres'] # 型態list
    getgenres2 = ','.join(getgenres) # 型態str
    
    list_artist_name = []
    list_artist_id = []
    list_artist_genres = []
    list_artist_name.append(getname)
    list_artist_id.append(getid)
    list_artist_genres.append(getgenres2)
    return list_artist_genres

In [9]:
# 輸出歌手名稱+Id+Genres的csv(artist_list_ID+Genres_版本.csv)
with open("csv/20200303/v3/artist_list_ID+Genres_v3.csv", "a", encoding="utf_8_sig", newline='') as csvfile:
    writer = csv.writer(csvfile)
    #第一次寫入:w，之後寫入:a
    #writer.writerow(['Artist','Id','Genres'])
    df_artists = pd.read_excel(r'csv/20200303/v3/artist_list_v3.xlsx')
    base = 1500
    df = df_artists[['Artist']][1500:] #base:base+100/最後一個100[400:]
    L_artist_name = []
    L_artist_id = []
    L_artist_genres = []
    no_item = []
    for i in range(len(df)):
        artist_name = df['Artist'][base+i]
        print(str(i) + '|' + str(artist_name))
        # 判斷check值是否為1(代表歌手不存在)
        check = get_artist(artist_name)
        if check != 1:         
            try:
                artist_info = get_artist(artist_name)
                artist_id = artist_info['id']
                #print(artist_id)
                list_artist_genres = get_artist_name_genres(access_token,artist_id)
                artist_genres = ",".join(list_artist_genres) # list >>>str
                L_artist_name.append(artist_name)
                L_artist_id.append(artist_id)
                L_artist_genres.append(list_artist_genres)
            except:
                L_artist_id.append('')
                L_artist_genres.append('')
                pass
            #之後寫入
            writer.writerow([artist_name, artist_id, artist_genres])
        else:
            no_item.append(artist_name)
            pass

0|Rachel Sweet
1|Raina Rose
2|Rajiv Dhall
3|Ramblin' Jack Elliott
4|Ramblin' Tommy Scott
5|Ramiele Malubay
6|Randy Crouch
7|Randy Howard
8|Randy Jackson
9|Randy Jones 
10|Randy Sparks
11|Raquel Castro
12|Raven-Symoné
13|Ray Korona
14|Ray LaMontagne
15|Ray Peterson
16|Ray Repp
17|Ray Simpson
18|Ray Smith
19|Ray Stephens 
20|Raye Zaragoza
21|Rebecca Black
22|Rebecca Loebe
23|Rebecca Schiffman
24|Red Grammer
25|Redfoo
26|Reggie Sears
None items
27|Regina Belle
28|Regina Zernay Roberts
None items
29|Renee Sands
30|Renee-Louise Carafice
31|Reverend Freakchild
32|Rex Smith
33|Rhett Miller
34|Rhys 
35|Rian Johnson
None items
36|Richard Dobson
37|Richard Dyer-Bennet
38|Richard Fariña
39|Richard Shindell
40|Richard Street
41|Richie Havens
42|Rick Shorter
43|Rick Spencer 
44|Rickie Lee Jones
45|Ricky Meléndez
46|Ricky Nelson
47|Riley Baugus
48|Rising Appalachia
49|Rita Abrams
50|Rita Coolidge
51|Rita Hosking
52|Rob Hegel
53|Robbie Fulks
retrying ...2secs
54|Robert A. Leonard
55|Robert German
56|

In [10]:
# 要保留的歌手類型清單  (透過getArtistGenres.ipnb整理)
df_genres_select = pd.read_excel(r'csv/20200303/artist_genres_select.xlsx')
df_genres_select.drop('Unnamed: 0',axis=1,inplace=True)
df_genres_select

,Genres
0,pop
1,dance pop
2,post-teen pop
3,urban contemporary
4,indie poptimism
...,...
1725,swedish choir
1726,cathedral choir
1727,atmosphere
1728,wind quintet


In [11]:
# 讀取artist_list_ID+Genres_版本.csv
df_id_genres = pd.read_csv(r'csv/20200303/v3/artist_list_ID+Genres_v3.csv')
df_id_genres

,Artist,Id,Genres
0,A Girl Called Eddy,4R024vmmAv1tyJGZPWkEDu,NaN
1,A. A. Bondy,4l8UbJGiXxbfyDJPvcVH23,"alternative country,chamber pop,indie folk,new..."
2,Aaliyah,0urTpYCsixqZwgNTkPJOJ4,"dance pop,hip hop,hip pop,pop,pop rap,r&b,urba..."
3,Aaron Lee Tasjan,4PztbfCny3X9gBjlpgvjYo,"alternative country,deep new americana,indie f..."
4,Abner Jay,0MPTLi41EZ3fWi3GYQvHQi,"garage punk blues,outsider,traditional folk"
...,...,...,...
1738,Zak Waters,0B8XBGloduGCBtRK8nm2YF,indie poptimism
1739,Zakiya,124AxXBDIQi7Ko1LGI6eUj,NaN
1740,Zella Day,100sLnojEpcadRx4edEBA6,"electropop,gauze pop,indie poptimism,indietron..."
1741,Zendaya,6sCbFbEjbYepqswM1vWjjs,"post-teen pop,viral pop"


In [12]:
# 刪除Genres為空的那一列
df_id_genres_drop = df_id_genres.dropna(axis=0, subset=['Genres'])
df_id_genres_drop.reset_index(inplace=True,drop=True)
df_id_genres_drop.to_csv('csv/20200303/v3/artist_list_ID+Genres_drop_v3.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop

,Artist,Id,Genres
0,A. A. Bondy,4l8UbJGiXxbfyDJPvcVH23,"alternative country,chamber pop,indie folk,new..."
1,Aaliyah,0urTpYCsixqZwgNTkPJOJ4,"dance pop,hip hop,hip pop,pop,pop rap,r&b,urba..."
2,Aaron Lee Tasjan,4PztbfCny3X9gBjlpgvjYo,"alternative country,deep new americana,indie f..."
3,Abner Jay,0MPTLi41EZ3fWi3GYQvHQi,"garage punk blues,outsider,traditional folk"
4,Abra Moore,4Nfj9qeKS8OmfFmOA6K6K2,"ectofolk,lilith,new wave pop"
...,...,...,...
923,Yulianna,09tP2EPNNwU20jaHUgXBG2,"russian dance pop,russian pop"
924,Yvonne Elliman,2d6JU9LvNhZR7AAtu4x2rS,disco
925,Zak Waters,0B8XBGloduGCBtRK8nm2YF,indie poptimism
926,Zella Day,100sLnojEpcadRx4edEBA6,"electropop,gauze pop,indie poptimism,indietron..."


In [13]:
# 逐行查看歌手所屬類別是否在篩選後的總類別中(artist_genres_select.xlsx)
count = 0
delete_index = []
df_id_genres_drop = pd.read_csv(r'csv/20200303/v3/artist_list_ID+Genres_drop_v3.csv')
for row in range(len(df_id_genres_drop)):
    # 把歌手類型清單的類型欄位轉換成list
    # https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_datafra
    List1 = df_genres_select['Genres'].tolist() # 總類別清單
    List2 = df_id_genres_drop['Genres'][row].split(',') # 每一位歌手的類別清單
    
    # 檢查歌手的類別有沒有包含在總類別中 (any:歌手任何一個類別不包含在總類別中就不刪除/all:歌手所有類別都在總類別中就刪除)
    check = all(item in List1 for item in List2)
    
    # 有包含
    if check is True:
        # 原始型態set，轉成list型態，印出有交集的類別
        # https://www.geeksforgeeks.org/python-convert-set-into-a-list/
        set1 = set(List1) 
        set2 = set(List2)
        print(str(count) + ' : ' + 'List1 ⊂ List2' + '\n' + 'Contain Genres:' + str(list(set1.intersection(set2))) + '\n' + '-'*90)
    # 沒包含
    else:
        print(str(count) + ' : ' + 'List1 ⊄ List2' + '\n' + '-'*90)
        delete_index.append(count)
        pass
    count += 1

print('Delete Index:',delete_index)

0 : List1 ⊄ List2
------------------------------------------------------------------------------------------
1 : List1 ⊄ List2
------------------------------------------------------------------------------------------
2 : List1 ⊄ List2
------------------------------------------------------------------------------------------
3 : List1 ⊄ List2
------------------------------------------------------------------------------------------
4 : List1 ⊄ List2
------------------------------------------------------------------------------------------
5 : List1 ⊂ List2
Contain Genres:['strut']
------------------------------------------------------------------------------------------
6 : List1 ⊄ List2
------------------------------------------------------------------------------------------
7 : List1 ⊂ List2
Contain Genres:['san antonio indie']
------------------------------------------------------------------------------------------
8 : List1 ⊄ List2
------------------------------------------------

429 : List1 ⊂ List2
Contain Genres:['arkansas country', 'banjo']
------------------------------------------------------------------------------------------
430 : List1 ⊄ List2
------------------------------------------------------------------------------------------
431 : List1 ⊄ List2
------------------------------------------------------------------------------------------
432 : List1 ⊂ List2
Contain Genres:['oklahoma country']
------------------------------------------------------------------------------------------
433 : List1 ⊄ List2
------------------------------------------------------------------------------------------
434 : List1 ⊄ List2
------------------------------------------------------------------------------------------
435 : List1 ⊄ List2
------------------------------------------------------------------------------------------
436 : List1 ⊄ List2
------------------------------------------------------------------------------------------
437 : List1 ⊄ List2
-----------

------------------------------------------------------------------------------------------
799 : List1 ⊄ List2
------------------------------------------------------------------------------------------
800 : List1 ⊄ List2
------------------------------------------------------------------------------------------
801 : List1 ⊄ List2
------------------------------------------------------------------------------------------
802 : List1 ⊂ List2
Contain Genres:['movie tunes']
------------------------------------------------------------------------------------------
803 : List1 ⊂ List2
Contain Genres:['brill building pop']
------------------------------------------------------------------------------------------
804 : List1 ⊄ List2
------------------------------------------------------------------------------------------
805 : List1 ⊄ List2
------------------------------------------------------------------------------------------
806 : List1 ⊄ List2
-------------------------------------------

In [14]:
# 篩選後的歌手列表，輸出csv(artist_list_ID+Genres_select_版本.csv)
# 若沒包含，則把歌手的那一列從dataframe(df_id_genres_drop)中刪除
# https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/
df_id_genres_drop.drop(df_id_genres_drop.index[[delete_index]],inplace=True)
df_id_genres_drop.reset_index(drop=True, inplace=True)
df_id_genres_drop.to_csv('csv/20200303/v3/artist_list_ID+Genres_select_v3.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop

D:\Anaconda3\lib\site-packages\pandas\core\indexes\base.py:4291: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


,Artist,Id,Genres
0,Adam Joseph,2Dymd4r1i4EfE8iooyCi1L,strut
1,Aisha Burns,1AZoAl5rMnZWGNPGr88krQ,san antonio indie
2,Akon,0z4gvV4rjIZ9wHck67ucSV,"dance pop,pop"
3,Alec Ounsworth,6Ej7FjLCm6jnolLCBmyRo6,stomp and holler
4,Alex Goot,66Fb5gJ9SX2WGlqDLUpjux,"channel pop,post-teen pop,viral pop"
...,...,...,...
317,Whitney Houston,6XpaIBNiVzIetEPCWDvAFP,"dance pop,pop,urban contemporary"
318,Will Oakland,7aoyGipPQaCvDTrJIe49W8,vaudeville
319,William Hung,3bU97PoqnxjKD45CrfwmC9,"outsider,talent show"
320,Zak Waters,0B8XBGloduGCBtRK8nm2YF,indie poptimism


# ---

In [53]:
# Test逐行查看歌手所屬類別是否在篩選後的總類別中(artist_genres_select.xlsx)
count2 = 0
delete_index2 = []
df_id_genres_drop2 = pd.read_csv(r'csv/20200303/v2/artist_list_ID+Genres_drop_v2test.csv')
df_id_genres_drop2

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Abraham Mateo,2bxxlINUlcMQQb39K7IopR,"latin,latin arena pop,latin pop,latin viral po..."
3,Adam Lambert,6prmLEyn4LfHlD9NnXWlf7,"dance pop,idol,pop,post-teen pop"
4,Adam Levine,4bYPcJP5jwMhSivRcqie2n,pop rock
...,...,...,...
337,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"
338,Willow Smith,3rWZHrfrsPBxVy692yAIxF,"afrofuturism,dance pop,electropop,pop"
339,Wizkid,3tVQdUvClmAT7URs9V3rsp,"afro dancehall,afropop,azonto,nigerian hip hop..."
340,Zach Herron,6sa5JC8xdrw2ZpGQ50nVCW,"social media pop,teen pop"


In [54]:
# Test
for row in range(len(df_id_genres_drop2)):
    # 把歌手類型清單的類型欄位轉換成list
    # https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_datafra
    List1_ = df_genres_select['Genres'].tolist() # 總類別清單
    List2_ = df_id_genres_drop2['Genres'][row].split(',') # 每一位歌手的類別清單
    
    # 檢查歌手的類別有沒有包含在總類別中 (any:歌手任何一個類別不包含在總類別中就不刪除/all:歌手所有類別都在總類別中就刪除)
    check2 = all(item in List1_ for item in List2_)
    
    # 有包含
    if check2 is True:
        # 原始型態set，轉成list型態，印出有交集的類別
        # https://www.geeksforgeeks.org/python-convert-set-into-a-list/
        set1_ = set(List1_) 
        set2_ = set(List2_)
        print(str(count2) + ' : ' + 'List1 ⊂ List2' + '\n' + 'Contain Genres:' + str(list(set1_.intersection(set2_))) + '\n' + '-'*90)
    # 沒包含
    else:
        print(str(count2) + ' : ' + 'List1 ⊄ List2' + '\n' + '-'*90)
        delete_index2.append(count2)
        pass
    count2 += 1

print('Delete Index:',delete_index2)

0 : List1 ⊂ List2
Contain Genres:['post-teen pop', 'dance pop', 'pop', 'boy band']
------------------------------------------------------------------------------------------
1 : List1 ⊂ List2
Contain Genres:['post-teen pop', 'dance pop']
------------------------------------------------------------------------------------------
2 : List1 ⊄ List2
------------------------------------------------------------------------------------------
3 : List1 ⊂ List2
Contain Genres:['post-teen pop', 'dance pop', 'pop', 'idol']
------------------------------------------------------------------------------------------
4 : List1 ⊄ List2
------------------------------------------------------------------------------------------
5 : List1 ⊄ List2
------------------------------------------------------------------------------------------
6 : List1 ⊄ List2
------------------------------------------------------------------------------------------
7 : List1 ⊂ List2
Contain Genres:['swedish pop', 'europop', 'clas

------------------------------------------------------------------------------------------
309 : List1 ⊄ List2
------------------------------------------------------------------------------------------
310 : List1 ⊄ List2
------------------------------------------------------------------------------------------
311 : List1 ⊂ List2
Contain Genres:['brill building pop']
------------------------------------------------------------------------------------------
312 : List1 ⊄ List2
------------------------------------------------------------------------------------------
313 : List1 ⊄ List2
------------------------------------------------------------------------------------------
314 : List1 ⊄ List2
------------------------------------------------------------------------------------------
315 : List1 ⊂ List2
Contain Genres:['operatic pop']
------------------------------------------------------------------------------------------
316 : List1 ⊄ List2
------------------------------------------

In [55]:
# Test篩選後的歌手列表，輸出csv(artist_list_ID+Genres_select_版本.csv)
# 若沒包含，則把歌手的那一列從dataframe(df_id_genres_drop)中刪除
# https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/
df_id_genres_drop2.drop(df_id_genres_drop2.index[[delete_index2]],inplace=True)
df_id_genres_drop2.reset_index(drop=True, inplace=True)
df_id_genres_drop2.to_csv('csv/20200303/v2/artist_list_ID+Genres_select_v2test.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop2

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Adam Lambert,6prmLEyn4LfHlD9NnXWlf7,"dance pop,idol,pop,post-teen pop"
3,Agnetha Faltskog,7fUtt9kVZOyn9LWy0JbDRI,"classic swedish pop,europop,swedish pop"
4,AJ Mitchell,6dn6x1XOng3LOAnfTjUn77,"pop,post-teen pop"
...,...,...,...
132,Tori Kelly,1vSN1fsvrzpbttOYGsliDr,"dance pop,neo mellow,pop,post-teen pop,viral pop"
133,Vanessa Hudgens,6G9bygHlCyPgNGxK2l3YdE,"dance pop,hollywood,post-teen pop,viral pop"
134,Victoria Beckham,54w1dJI7MTjkRGNfGuu2fc,europop
135,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"


In [154]:
# 篩選後的歌手列表，輸出csv(artist_list_ID+Genres_select_版本.csv)
# 若沒包含，則把歌手的那一列從dataframe(df_id_genres_drop)中刪除
 # https://chrisalbon.com/python/data_wrangling/pandas_dropping_column_and_rows/
df_id_genres_drop.drop(df_id_genres_drop.index[[delete_count]],inplace=True)
df_id_genres_drop.reset_index(drop=True, inplace=True)
df_id_genres_drop.to_csv('csv/20200303/v2/artist_list_ID+Genres_select_v2.csv',encoding="utf_8_sig",index=0)
df_id_genres_drop

,Artist,Id,Genres
0,5 Seconds of Summer,5Rl15oVamLq7FbSb0NNBNy,"boy band,dance pop,pop,post-teen pop"
1,Aaron Carter,5338fdSzGNOFD4cxAxDTeq,"dance pop,post-teen pop"
2,Adam Rickitt,2u1BQaJtganAtbdBrUPZe3,bubblegum dance
3,Agnetha Faltskog,7fUtt9kVZOyn9LWy0JbDRI,"classic swedish pop,classic uk pop,europop,new..."
4,Agnez Mo,6pMR8Zgot664613rAiLC2Z,indonesian pop
...,...,...,...
322,Why Don't We,2jnIB6XdLvnJUeNTy5A0J2,"boy band,pop,post-teen pop,teen pop"
323,Willow Smith,3rWZHrfrsPBxVy692yAIxF,"afrofuturism,dance pop,electropop,pop,post-tee..."
324,Wizkid,3tVQdUvClmAT7URs9V3rsp,"afro dancehall,afropop,azonto,nigerian hip hop..."
325,Zach Herron,6sa5JC8xdrw2ZpGQ50nVCW,"social media pop,teen pop"


In [56]:
# 把歌手類型清單的類型欄位轉換成list
# https://www.interviewqs.com/ddi_code_snippets/get_list_pandas_dataframe
List1 = df_genres_select['Genres'].tolist()
List2 = df_test['Genres'][0].split(',')
#print('List1 =',List1)
print(len(List1))
#print('List2 =',List2)
print(len(List2))

if len(List1) == len(df_genres_select):
    print('Same')

check =  any(item in List1 for item in List2)
if check is True:
    print("List1 ⊂ List2")    
else :
    print("List1 ⊄ List2")

1730
4
Same
The list List1 contains some elements of the list List2


In [60]:
# list取交集的部分
def list_contains(List1, List2):  
    set1 = set(List1) 
    set2 = set(List2) 
    if set1.intersection(set2): 
        print('Contain Genres:',list(set1.intersection(set2))) # 原始型態set
        return True 
    else: 
        return False

List1 = df_genres_select['Genres'].tolist()
List2 = df_test['Genres'][0].split(',')
list_contains(List1, List2)

Contain Genres: ['pop', 'post-teen pop', 'boy band', 'dance pop']


True